In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = catalog.search(collections=["io-lulc-9-class"], bbox=mendocino_bbox)

items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
import stackstac

# The STAC metadata contains some information we'll want to use when creating
# our merged dataset. Get the EPSG code of the first item and the nodata value.
item = items[0]

# Create a single DataArray from out multiple resutls with the corresponding
# rasters projected to a single CRS. Note that we set the dtype to ubyte, which
# matches our data, since stackstac will use float64 by default.
stack = (
    stackstac.stack(
        items,
        dtype=float,
        fill_value=255,
        bounds_latlon=mendocino_bbox,
        sortby_date=False,
        epsg=4326
    )
    .assign_coords(
        time=pd.to_datetime([item.properties["start_datetime"] for item in items])
        .tz_convert(None)
        .to_numpy()
    )
    .sortby("time")
)

stack

In [ ]:
import dask.distributed

client = dask.distributed.Client(processes=False)
print(f"/proxy/{client.scheduler_info()['services']['dashboard']}/status")

In [ ]:
# Filter the stack by year before computing
stack_filtered = stack.sel(time=stack.time.dt.year.isin([2017, 2019]))

print(f"Original stack shape: {stack.shape}")
print(f"Filtered stack shape: {stack_filtered.shape}")

# Now compute only the filtered data
merged = stack_filtered.squeeze().compute()

print(f"Final merged data shape: {merged.shape}")
print(f"Years in merged data: {merged.time.dt.year.values}")

In [ ]:
# g = merged.plot(col="time")
# for ax in g.axs.flat:
#     ax.set_axis_off()


In [ ]:
from pystac.extensions.item_assets import ItemAssetsExtension

collection = catalog.get_collection("io-lulc-9-class")
ia = ItemAssetsExtension.ext(collection)

x = ia.item_assets["data"]
class_names = {x["summary"]: x["values"][0] for x in x.properties["file:values"]}
values_to_classes = {v: k for k, v in class_names.items()}
class_count = len(class_names)
class_names

In [ ]:
with rasterio.open(item.assets["data"].href) as src:
    colormap_def = src.colormap(1)  # get metadata colormap for band 1
    colormap = [
        np.array(colormap_def[i]) / 255 for i in range(max(class_names.values()) + 1)
    ]  # transform to matplotlib color format

cmap = ListedColormap(colormap)

In [ ]:
vmin = 0
vmax = max(class_names.values()) + 1
epsg = merged.epsg.item()
# Convert to a standard projection before plotting
merged = merged.rio.reproject("EPSG:4326")

p = merged.plot(
    subplot_kws=dict(projection=ccrs.PlateCarree()),
    col="time",
    transform=ccrs.PlateCarree(),
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    figsize=(16, 6),
)
ticks = np.linspace(0.5, vmax - 0.5, vmax - vmin)
labels = [values_to_classes.get(i, "") for i in range(cmap.N)]
p.cbar.set_ticks(ticks, labels=labels)
p.cbar.set_label("Class")

In [ ]:
colors = list(cmap.colors)

ax = (
    pd.value_counts(merged.data.ravel(), sort=False)
    .sort_index()
    .reindex(range(cmap.N), fill_value=0)
    .rename(values_to_classes)
    .plot.barh(color=colors, rot=0, width=0.9)
)
ax.set(
    title="Distribution of Land Cover classes",
    ylabel="Landcover class",
    xlabel="Class count",
)

In [ ]:
# Plot the land cover summary statistics
plt.figure(figsize=(10, 6))

ax = sns.barplot(data=summary_df,
                y='land_cover_type',  
                x='total_percentage',
                color='#40A135',
                edgecolor='black',
                orient='h'
                )

# Add values on top of the bars
for container in ax.containers:
    ax.bar_label(container, padding=5, fmt='%.1f%%')  

# Format x-axis tick labels
ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=100, decimals=0))
ax.set_axisbelow(True) # Set below the bars
ax.xaxis.grid(True, which='major', linestyle='--', color='lightgrey')

# Remove plot borders for clarity
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Title and labels
ax.set_title('Distribution of Land Cover within the Ranch Fire Perimeter\n(2011 USGS National Terrestrial Ecosystems Data)', fontsize=13)
ax.set_xlabel('')
ax.set_ylabel('')
plt.yticks(fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# Get the raster resolution in meters
pixel_width = abs(lulc_clip_utm.rio.resolution()[0])
pixel_height = abs(lulc_clip_utm.rio.resolution()[1])
pixel_area = pixel_width * pixel_height

# Convert pixel area from m2 to km2
pixel_area_km2 = pixel_area / 1_000_000 

# Define land cover types to search for
lc_types = ['Woodland', 'Savanna', 'Chaparral', 'Grassland', 'Developed']

# Create summary dataframe
summary_data = []

for lc in lc_types:
    # Filter classes containing the land cover type
    filtered_classes = valid_classes[valid_classes['class_label'].str.contains(lc, case=False, na=False)]
    
    # Calculate totals 
    total_pixels = filtered_classes['pixel_count'].sum()
    total_area = total_pixels * pixel_area_km2
    total_percent = filtered_classes['percentage'].sum()
    
    # Append data
    summary_data.append({
        'land_cover_type': lc,
        'total_percentage': total_percent,
        'total_area_km2': total_area,
        'total_pixels': total_pixels,
        'num_classes': len(filtered_classes)
    })

# Convert to DataFrame and sort
summary_df = pd.DataFrame(summary_data).sort_values('total_percentage', ascending=False)

print("Summary of Land Cover Types:")
print(summary_df)

In [ ]:
# Create a boxplot of RBR by major vegetation types
plt.figure(figsize=(12, 8))

# Filter to top 1 most common vegetation types for clarity
top_types = significant_types.head(10).index
plot_data = combined_analysis[combined_analysis['class_label'].isin(top_types)].sort_values('rbr', ascending=False)

sns.boxplot(data=plot_data, y='class_label', x='rbr', orient='h', color='#40A135')
plt.title('Distribution of Relative Burn Ratio by Vegetation Type\nMendocino Complex Fire')
plt.xlabel('Relative Burn Ratio (RBR)')
plt.ylabel('Vegetation Type')
plt.tight_layout()
plt.show()